In [1]:
import networkx as nx
neighbors_fpath = "model/text8.graph"
G = nx.read_edgelist(neighbors_fpath, nodetype=str, delimiter="\t", data=(('weight',float),))

In [6]:
x??

In [6]:
import codecs 
from multiprocessing import Pool


def get_ego_network(ego):
    res = "{}\t{}\t{}\n".format(ego, len(G.nodes), len(G[ego]))
    return res

output_fpath = "model/tmp.txt"
num_cores = 40
with codecs.open(output_fpath, "w", "utf-8") as output, Pool(num_cores) as pool:
    for j in range(100):
        for i, res in enumerate(pool.imap_unordered(get_ego_network, G.nodes)):
            output.write(res)

Process ForkPoolWorker-79:
Process ForkPoolWorker-80:
Process ForkPoolWorker-75:
Process ForkPoolWorker-53:
Process ForkPoolWorker-77:
Process ForkPoolWorker-76:
Process ForkPoolWorker-56:
Process ForkPoolWorker-63:
Process ForkPoolWorker-55:
Process ForkPoolWorker-69:
Process ForkPoolWorker-54:
Process ForkPoolWorker-58:
Process ForkPoolWorker-62:
Process ForkPoolWorker-60:
Process ForkPoolWorker-67:
Process ForkPoolWorker-49:
Process ForkPoolWorker-65:
Process ForkPoolWorker-41:
Process ForkPoolWorker-68:
Process ForkPoolWorker-50:
Process ForkPoolWorker-61:
Process ForkPoolWorker-57:
Process ForkPoolWorker-66:
Process ForkPoolWorker-70:
Process ForkPoolWorker-48:
Process ForkPoolWorker-52:
Process ForkPoolWorker-78:
Process ForkPoolWorker-44:
Process ForkPoolWorker-59:
Traceback (most recent call last):
Process ForkPoolWorker-47:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most

  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/panchenko/anaconda/l

  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/synch

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
Traceback (most recent call last):
KeyboardInterrupt
Traceback (most recent call last):
KeyboardInterrupt
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
KeyboardInterrupt
KeyboardInterrupt
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run

KeyboardInterrupt: 